# Legacy Macrostrat Enricher

In [32]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
import ast

# ------------------------------------------------------------
# Map-Only Approach
# ------------------------------------------------------------

def safe_get_map(url, params=None, timeout=12):
    try:
        r = requests.get(url, params=params, timeout=timeout)
        if r.status_code == 200:
            return r.json()
        return None
    except Exception as e:
        return None

def fetch_macrostrat_map_only(lat, lon):
    """Fetch geological data using only the /geologic_units/map endpoint"""
    url = "https://macrostrat.org/api/v2/geologic_units/map"
    params = {
        "lat": lat,
        "lng": lon,
        "scale": "medium"
    }

    js = safe_get_map(url, params)
    if not js or not js.get("success"):
        return None

    units = js.get("success", {}).get("data", [])
    if not units:
        return None

    # Take the first (surface) unit
    unit = units[0]
    
    # Handle lithology - can be string, list of strings, or list of dicts
    lith = unit.get("lith", [])
    if isinstance(lith, list):
        if lith and isinstance(lith[0], dict):
            # Extract 'name' field from dict objects
            lith_names = [item.get("name") for item in lith if isinstance(item, dict) and item.get("name")]
            lithology = ", ".join(lith_names)
        else:
            # List of strings
            lithology = ", ".join(str(item) for item in lith)
    else:
        lithology = str(lith) if lith else None
    
    return {
        "MACRO_UNIT_NAME": unit.get("name"),
        "MACRO_STRAT_NAME": unit.get("strat_name"),
        "MACRO_LITHOLOGY": lithology,
        "MACRO_AGE_MIN": unit.get("t_int_age"),
        "MACRO_AGE_MAX": unit.get("b_int_age"),
        "MACRO_DESCRIPTION": unit.get("descrip"),
        "MACRO_COLOR": unit.get("color"),
        "MACRO_SOURCE_ID": unit.get("source_id"),
        "MACRO_MAP_ID": unit.get("map_id")
    }

def enrich_macrostrat_map_only(
    coord_csv_path,
    output_csv_path,
    sample_n=None,
    sleep_sec=0.1
):
    """Map-only enrichment using /geologic_units/map endpoint"""
    
    # Load coordinates
    df = pd.read_csv(coord_csv_path)
    
    def parse_coord(x):
        try:
            return ast.literal_eval(x) if isinstance(x, str) else x
        except:
            return None

    df["COORDINATES"] = df["COORDINATES"].apply(parse_coord)
    df = df[df["COORDINATES"].notna()].copy()

    # Apply sampling
    if sample_n is not None:
        df = df.head(sample_n)

    rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Macrostrat Map"):
        
        sid = row["STRUCTURE_ID"]
        lat, lon = row["COORDINATES"]

        entry = {
            "STRUCTURE_ID": sid,
            "COORDINATES": (lat, lon),
            "MACRO_UNIT_NAME": None,
            "MACRO_STRAT_NAME": None,
            "MACRO_LITHOLOGY": None,
            "MACRO_AGE_MIN": None,
            "MACRO_AGE_MAX": None,
            "MACRO_DESCRIPTION": None,
            "MACRO_COLOR": None,
            "MACRO_SOURCE_ID": None,
            "MACRO_MAP_ID": None
        }

        result = fetch_macrostrat_map_only(lat, lon)
        if result:
            entry.update(result)

        rows.append(entry)
        time.sleep(sleep_sec)

    # Save output
    df_out = pd.DataFrame(rows)
    df_out.to_csv(output_csv_path, index=False)

    # Show summary
    non_null_count = df_out['MACRO_UNIT_NAME'].notna().sum()
    print(f"Completed: {non_null_count}/{len(rows)} structures enriched")
    
    return df_out

# Run the map-only version
enrich_macrostrat_map_only(
    coord_csv_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv",
    output_csv_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\macrostrat_map_only.csv",
    sample_n=None
)

Macrostrat Map: 100%|██████████| 4914/4914 [1:08:44<00:00,  1.19it/s]

Completed: 4754/4914 structures enriched


,STRUCTURE_ID,COORDINATES,MACRO_UNIT_NAME,MACRO_STRAT_NAME,MACRO_LITHOLOGY,MACRO_AGE_MIN,MACRO_AGE_MAX,MACRO_DESCRIPTION,MACRO_COLOR,MACRO_SOURCE_ID,MACRO_MAP_ID
0,,"(47.98571667, -122.2271222)",None,None,None,NaN,NaN,None,None,NaN,NaN
1,,"(47.697975, -122.6195)",Younger glacial drift,Vashon Drift; Sumas Drift,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Till. Hard, blue-gray to gray concrete-like m...",#F9F97F,133.0,2752774.0
2,,"(48.21215, -121.9331306)",Older glacial drift,Admiralty Clay; Orting Drift; Puyallup Formati...,"Major:{fine alluvium,coarse alluvium}, Inciden...",0.0117,3.60,"Till, outwash, and associated interglacial dep...",#FFF0BD,133.0,2717109.0
3,,"(47.56759167, -122.5517028)",Younger glacial drift,Vashon drift; Possession Drift; Whidbey Formation,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Younger glacial drift, undivided. Till, outwa...",#FFF2C7,133.0,2815013.0
4,,"(47.769275, -122.707925)",Younger glacial drift,Vashon Drift; Sumas Drift,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Till. Hard, blue-gray to gray concrete-like m...",#F9F97F,133.0,2752774.0
...,...,...,...,...,...,...,...,...,...,...,...
4909,DAPFORLE,"(47.02788056, -122.5136)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4910,DAPFORLE,"(47.02893056, -122.4655)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4911,DAPFORLE,"(47.11668889, -122.5003)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4912,DAPFORLE,"(47.10798889, -122.5898)",water,,None,NaN,NaN,,,133.0,2764941.0


# Upgraded Macrostrat Enricher

In [2]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
import ast
from concurrent.futures import ThreadPoolExecutor, as_completed

# ------------------------------------------------------------
# IMPROVED Map-Only Approach with Parallel Processing
# ------------------------------------------------------------

def safe_get_map(url, params=None, timeout=12):
    try:
        r = requests.get(url, params=params, timeout=timeout)
        if r.status_code == 200:
            return r.json()
        return None
    except Exception as e:
        return None

def fetch_macrostrat_map_only(lat, lon):
    """Fetch geological data using only the /geologic_units/map endpoint"""
    url = "https://macrostrat.org/api/v2/geologic_units/map"
    params = {
        "lat": lat,
        "lng": lon,
        "scale": "medium"
    }

    js = safe_get_map(url, params)
    if not js or not js.get("success"):
        return None

    units = js.get("success", {}).get("data", [])
    if not units:
        return None

    # Take the first (surface) unit
    unit = units[0]
    
    # Handle lithology - can be string, list of strings, or list of dicts
    lith = unit.get("lith", [])
    if isinstance(lith, list):
        if lith and isinstance(lith[0], dict):
            # Extract 'name' field from dict objects
            lith_names = [item.get("name") for item in lith if isinstance(item, dict) and item.get("name")]
            lithology = ", ".join(lith_names)
        else:
            # List of strings
            lithology = ", ".join(str(item) for item in lith)
    else:
        lithology = str(lith) if lith else None
    
    return {
        "MACRO_UNIT_NAME": unit.get("name"),
        "MACRO_STRAT_NAME": unit.get("strat_name"),
        "MACRO_LITHOLOGY": lithology,
        "MACRO_AGE_MIN": unit.get("t_int_age"),
        "MACRO_AGE_MAX": unit.get("b_int_age"),
        "MACRO_DESCRIPTION": unit.get("descrip"),
        "MACRO_COLOR": unit.get("color"),
        "MACRO_SOURCE_ID": unit.get("source_id"),
        "MACRO_MAP_ID": unit.get("map_id")
    }

def process_single_row(row_data):
    """Process a single row - designed for parallel execution"""
    sid, lat, lon = row_data
    
    entry = {
        "STRUCTURE_ID": sid,
        "COORDINATES": (lat, lon),
        "MACRO_UNIT_NAME": None,
        "MACRO_STRAT_NAME": None,
        "MACRO_LITHOLOGY": None,
        "MACRO_AGE_MIN": None,
        "MACRO_AGE_MAX": None,
        "MACRO_DESCRIPTION": None,
        "MACRO_COLOR": None,
        "MACRO_SOURCE_ID": None,
        "MACRO_MAP_ID": None
    }
    
    result = fetch_macrostrat_map_only(lat, lon)
    if result:
        entry.update(result)
    
    return entry

def enrich_macrostrat_parallel(
    coord_csv_path,
    output_csv_path,
    sample_n=None,
    max_workers=10
):
    """
    Parallel enrichment using ThreadPoolExecutor.
    
    Args:
        coord_csv_path: Path to coordinates CSV
        output_csv_path: Path for output CSV
        sample_n: Number of samples to process (None = all)
        max_workers: Number of parallel threads (default 10)
                    Higher = faster, but don't overload the API
    """
    
    # Load coordinates
    df = pd.read_csv(coord_csv_path)
    
    def parse_coord(x):
        try:
            return ast.literal_eval(x) if isinstance(x, str) else x
        except:
            return None

    df["COORDINATES"] = df["COORDINATES"].apply(parse_coord)
    df = df[df["COORDINATES"].notna()].copy()

    # Apply sampling
    if sample_n is not None:
        df = df.head(sample_n)

    # Prepare data for parallel processing
    tasks = [(row["STRUCTURE_ID"], row["COORDINATES"][0], row["COORDINATES"][1]) 
             for _, row in df.iterrows()]
    
    print(f"Processing {len(tasks)} structures with {max_workers} parallel workers...")
    
    rows = []
    
    # Use ThreadPoolExecutor for parallel API calls
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_task = {executor.submit(process_single_row, task): task for task in tasks}
        
        # Process completed tasks with progress bar
        for future in tqdm(as_completed(future_to_task), total=len(tasks), desc="Macrostrat Parallel"):
            try:
                result = future.result()
                rows.append(result)
            except Exception as e:
                # If a task fails, add empty entry
                task = future_to_task[future]
                sid, lat, lon = task
                rows.append({
                    "STRUCTURE_ID": sid,
                    "COORDINATES": (lat, lon),
                    "MACRO_UNIT_NAME": None,
                    "MACRO_STRAT_NAME": None,
                    "MACRO_LITHOLOGY": None,
                    "MACRO_AGE_MIN": None,
                    "MACRO_AGE_MAX": None,
                    "MACRO_DESCRIPTION": None,
                    "MACRO_COLOR": None,
                    "MACRO_SOURCE_ID": None,
                    "MACRO_MAP_ID": None
                })

    # Save output
    df_out = pd.DataFrame(rows)
    df_out.to_csv(output_csv_path, index=False)

    # Show summary
    non_null_count = df_out['MACRO_UNIT_NAME'].notna().sum()
    print(f"Completed: {non_null_count}/{len(rows)} structures enriched")
    print(f"Success rate: {non_null_count/len(rows)*100:.1f}%")
    
    return df_out

# Run the parallel version (MUCH FASTER!)
enrich_macrostrat_parallel(
    coord_csv_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv",
    output_csv_path=r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data_fixed\macrostrat.csv",
    sample_n=None,
    max_workers=10  # Adjust this based on API limits - 10-20 is usually good
)

Processing 4914 structures with 10 parallel workers...


Macrostrat Parallel: 100%|██████████| 4914/4914 [10:58<00:00,  7.46it/s]

Completed: 4754/4914 structures enriched
Success rate: 96.7%


,STRUCTURE_ID,COORDINATES,MACRO_UNIT_NAME,MACRO_STRAT_NAME,MACRO_LITHOLOGY,MACRO_AGE_MIN,MACRO_AGE_MAX,MACRO_DESCRIPTION,MACRO_COLOR,MACRO_SOURCE_ID,MACRO_MAP_ID
0,00000000,"(47.25279444, -124.178075)",Terrace deposits,Montesano Formation,"Major:{gravel,sand}, Minor:{clay,silt}",0.0117,2.58,Unconsolidated to partly consolidated fluvial ...,#FFFF00,133.0,2753122.0
1,,"(47.98571667, -122.2271222)",None,None,None,NaN,NaN,None,None,NaN,NaN
2,,"(47.56759167, -122.5517028)",Younger glacial drift,Vashon drift; Possession Drift; Whidbey Formation,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Younger glacial drift, undivided. Till, outwa...",#FFF2C7,133.0,2815013.0
3,,"(47.769275, -122.707925)",Younger glacial drift,Vashon Drift; Sumas Drift,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Till. Hard, blue-gray to gray concrete-like m...",#F9F97F,133.0,2752774.0
4,00000000,"(47.24963889, -124.1740556)",Terrace deposits,Montesano Formation,"Major:{gravel,sand}, Minor:{clay,silt}",0.0117,2.58,Unconsolidated to partly consolidated fluvial ...,#FFFF00,133.0,2753122.0
...,...,...,...,...,...,...,...,...,...,...,...
4909,DAPFORLE,"(47.02788056, -122.5136)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4910,DAPFORLE,"(47.11668889, -122.5003)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4911,DAPFORLE,"(47.10723889, -122.5479)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
4912,DAPFORLE,"(47.02893056, -122.4655)",Younger glacial drift,Vashon Drift; Steilacoom Gravel; Washtucna Gra...,"Major:{fine alluvium,coarse alluvium}",0.0117,2.58,"Advance and recessional outwash, stratified dr...",#FFF2AE,133.0,2718478.0
